### Importing classes and functions

In [ ]:
import numpy
import sys
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

### Load ASCII test for the book and convert all characters into lowercase to reduce the vocabulary

In [ ]:
filename = "wonderland.txt"
raw_text = open(filename).read()
raw_text = raw_text.lower()

### Create mapping of unique characters in the book to unique integers

In [ ]:
chars = sorted(list(set(raw_text)))
char_to_int = dict ((c,i) for i,c in enumerate(chars))

### Summarizing the dataset

In [ ]:
n_chars = len(raw_text)
n_vocab = len (chars)
print ("Total Charcters: ", n_chars)
print ("Total Vocab: ", n_vocab)

### Breaking up the text into subsequences with a fixed length of 100 characters and prepare the dataset of input to output pairs encoded as integers to be exposed to the network for training

In [ ]:
seq_length = 100 
dataX = []
dataY = []
for i in range (0, n_chars - seq_length, 1):
    seq_in = raw_text [i : i + seq_length]
    seq_out = raw_text [i + seq_length]
    dataX.append ([char_to_int[char] for char in seq_in])
    dataY.append (char_to_int[seq_out])
n_patterns = len(dataX)
print ("Total Patterns for training: ", n_patterns)

### Transforming the list of input sequences into the form [samples, time steps, features]

In [ ]:
# reshape X 
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical (dataY)

### Defining the LSTM with single hidden LSTM layer with 256 memory units

In [ ]:
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

### Defining the checkpoint

In [ ]:
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

model.fit(X, y, epochs=20, batch_size=128, callbacks=callbacks_list)

### Generating text with the trained network

In [ ]:
# load the network weights
filename = "weights-improvement-20-2.0174.hdf5"
model.load_weights(filename)
model.compile( loss='categorical_crossentropy', optimizer = 'adam' )

int_to_char = dict ((i, c) for i,c in enumerate(chars))
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print ("Seed:")
print ("\"",''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters
for i in range (1000):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x/ float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_char [index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print ("\nDone.")